<h3> Leetcode Hard Problem 4 | User Purchase Platform | Complex SQL 11 </h3>

In [0]:
%sql
create table spending 
(
user_id int,
spend_date date,
platform varchar(10),
amount int
);

insert into spending values(1,'2019-07-01','mobile',100),(1,'2019-07-01','desktop',100),(2,'2019-07-01','mobile',100)
,(2,'2019-07-02','mobile',100),(3,'2019-07-01','desktop',100),(3,'2019-07-02','desktop',100);

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
select * from spending

user_id,spend_date,platform,amount
1,2019-07-01,mobile,100
1,2019-07-01,desktop,100
2,2019-07-01,mobile,100
2,2019-07-02,mobile,100
3,2019-07-01,desktop,100
3,2019-07-02,desktop,100


<h3> Solution in sql(hive) </h3>

In [0]:
%sql 
with cte as(
select spend_date,sum(amount)amount,count(distinct user_id)user_count,
case when count(user_id)>1 then 'both' else max(platform) end as platform
 from spending
 group by spend_date,user_id
 UNION
SELECT spend_date, 0 as Amount, 0 as Users_Count,'both' AS PlatformName
FROM spending
GROUP BY  spend_date)
select spend_date,platform,sum(amount)as total_amount,sum(user_count)as total_user from cte
group by spend_date,platform
order by spend_date,platform desc

spend_date,platform,total_amount,total_user
2019-07-01,mobile,100,1
2019-07-01,desktop,100,1
2019-07-01,both,200,1
2019-07-02,mobile,100,1
2019-07-02,desktop,100,1
2019-07-02,both,0,0


<h3> Solution in pyspark </h3>

In [0]:
from pyspark.sql.functions import col,countDistinct,sum,lit,count,max
from pyspark.sql.window import Window

In [0]:
df_s=spark.sql("select * from spending")
df_s.show()

+-------+----------+--------+------+
|user_id|spend_date|platform|amount|
+-------+----------+--------+------+
|      1|2019-07-01|  mobile|   100|
|      1|2019-07-01| desktop|   100|
|      2|2019-07-01|  mobile|   100|
|      2|2019-07-02|  mobile|   100|
|      3|2019-07-01| desktop|   100|
|      3|2019-07-02| desktop|   100|
+-------+----------+--------+------+



In [0]:
df_t=df_s.groupBy("spend_date","user_id").agg(sum(col("amount")).alias("amt"),\
    countDistinct("user_id").alias("tot_user"),count("user_id").alias("cnt"),max("platform").alias("platform1"))
df_p=df_t.withColumn("platform",when(col("cnt")>1,"both").otherwise(col("platform1")))
df_f=df_p.select("spend_date","amt","tot_user","platform").unionAll(df_s.groupBy("spend_date").\
    agg(lit(0).alias("amt"),lit(0).alias("tot_user"),lit("both").alias("platform")))
df_f.groupBy("spend_date","platform").agg(sum("amt").alias("total_amount"),sum("tot_user").alias("total_user"))\
    .orderBy("spend_date",col("platform").desc()).show()


+----------+--------+------------+----------+
|spend_date|platform|total_amount|total_user|
+----------+--------+------------+----------+
|2019-07-01|  mobile|         100|         1|
|2019-07-01| desktop|         100|         1|
|2019-07-01|    both|         200|         1|
|2019-07-02|  mobile|         100|         1|
|2019-07-02| desktop|         100|         1|
|2019-07-02|    both|           0|         0|
+----------+--------+------------+----------+

